In [3]:
# handle bugs in some docker container: 'pip install -e .'  doesn't work 
from IPython.display import clear_output
try:
    import Flamingo
except ModuleNotFoundError:
    import sys 
    sys.path.append('..')
    import Flamingo
lang_encoder_path = "facebook/opt-125m"
tokenizer_path = lang_encoder_path
cache_dir = None 
model_config = dict(
    clip_vision_encoder_path="ViT-L-14",
    clip_vision_encoder_pretrained="openai",
    lang_encoder_path=lang_encoder_path,
    tokenizer_path=tokenizer_path,
    cross_attn_every_n_layers=1,
    cache_dir = cache_dir,
    lora_tuning=False,
    decoupled=True  
)

import torch
from Flamingo.lora_tuning import create_model_and_transforms 
from Flamingo.models.batchprocessor import DecoupledFlamingoBatchProcessor
from Flamingo.datasets.gtsrb import classes
from Flamingo.config.participants_property import dataset_config
from Flamingo.datasets import build_dataset
from Flamingo.inference.vis import show_pred_with_gt
from Flamingo.inference import post_process
import pandas as pd 
import re
clear_output()

In [4]:
# build model, image processor and tokenizer
model, image_processor, tokenizer = create_model_and_transforms(
    **model_config
)
print("Load state dict:")
state_dict = torch.load("/root/yunzhi/flamingo_retrieval/retrieval/work_dir/99/weight.pth")
keys1 = model.lang_encoder.gated_cross_attn_layers.load_state_dict(state_dict, strict=False)
keys2 = model.perceiver.load_state_dict(state_dict, strict=False)
dataset = build_dataset(
    dataset_config=dataset_config,
    vis_processor=image_processor,
    tokenizer=tokenizer)
model = model.to(device="cuda:2", dtype=torch.bfloat16)

batch_processor = DecoupledFlamingoBatchProcessor(cast_type='bf16', tokenizer=tokenizer)
clear_output()

In [6]:
model.eval()
clear_output()

In [10]:
[k for k in dataset[0]]

['vision_x',
 'category_name',
 'attributes_name',
 'ori_img_name',
 'file_name',
 'bbox',
 'area',
 'pth_file']

In [12]:
for data in dataset:
    bbox = data['bbox']
    x, y, w, h = bbox
    x, y, w, h = int(x), int(y), int(w), int(h)
    if x <0 or y < 0 or w < 0 or h < 0:
        continue 
    break 
data['bbox'], data['category_name'], data['attributes_name'], data['vision_x'].shape

([110.2237445779, 294.2411995329, 81.59200867539998, 116.00141808469999],
 'pedestrian',
 '0-50 obstruction',
 torch.Size([1, 1, 1, 256, 1024]))

In [29]:
data = dataset[5000]

In [30]:
# data 

In [31]:

with torch.inference_mode():
    output = batch_processor(model=model, batch=data['vision_x'], mode='test',
            text_prompt="<image>Output:", num_beams=3, max_new_tokens=20)
    print("before post process:", output)
    # output = post_process(output, cats=cats)
output

before post process: ['Output:50-80 obstruction motorcycle electric vehicle (BEV),” the motorcycle electric vehicle (BEV']


['Output:50-80 obstruction motorcycle electric vehicle (BEV),” the motorcycle electric vehicle (BEV']

In [8]:
dataset[1]

{'vision_x': tensor([[[[[-0.6464, -1.5234, -0.2136,  ...,  0.7257, -0.1189, -0.2867],
            [ 0.4762,  0.5485,  0.0902,  ..., -0.1007,  0.1825, -0.4409],
            [ 0.5197,  0.9373,  0.1619,  ..., -0.0025,  0.5712, -0.3488],
            ...,
            [ 0.7610,  0.9900,  0.4876,  ...,  0.8991,  0.3367,  0.4553],
            [ 0.2455,  1.8385, -0.3149,  ...,  1.2226, -0.5920,  0.3794],
            [ 0.6058, -0.0835,  0.1039,  ...,  0.3892, -0.0593,  0.2578]]]]]),
 'category_name': 'pedestrian',
 'attributes_name': '0-50 obstruction',
 'ori_img_name': '075e8be308be5bd67db1251ffd335134.jpg',
 'file_name': '075e8be308be5bd67db1251ffd335134.jpg_1.pth',
 'bbox': [110.2237445779,
  294.2411995329,
  81.59200867539998,
  116.00141808469999],
 'area': 9464.788710725543,
 'pth_file': '/root/datasets/cached_participants_property/pth/pedestrian/075e8be308be5bd67db1251ffd335134.jpg_1.pth'}